In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
import networkx as nx
from rt_bundled_ego_chord_diagram import RTBundledEgoChordDiagram
import rtsvg
rt = rtsvg.RACETrack()
'''
edges_filename  = '../../data/stanford/facebook/1684.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = _split_[0], _split_[1]
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df = pl.DataFrame(_lu_)
# Only Keep The Largest Component
g = rt.createNetworkXGraph(df, [('fm','to')])
comp_largest = None
for _comp_ in nx.connected_components(g):
    if comp_largest is None:            comp_largest = _comp_
    if len(_comp_) > len(comp_largest): comp_largest = _comp_
df = df.filter(pl.col('fm').is_in(comp_largest) & pl.col('to').is_in(comp_largest))
_relates_ = [('fm','to')]
'''
import os
_dir_   = '../../data/2013_vast_challenge/mc3_netflow/nf'
_files_ = os.listdir(_dir_)
_dfs_   = []
for _file_ in _files_:
    _dfs_.append(pl.read_csv(_dir_ + '/' + _file_))
df = pl.concat(_dfs_)
_relates_ = [('firstSeenSrcIp', 'firstSeenDestIp')]
df.shape

In [ ]:
becd = RTBundledEgoChordDiagram(rt, df, _relates_, high_degree_node_count=10, chord_diagram_points=8, w=1600, h=1600)
rt.table([becd, becd.outlineSVG()], spacer=10, per_row=1)

In [ ]:
def pivot(_lu_):
    df_lu = {'bin':[],'__count__':[]}
    for k in _lu_: df_lu['bin'].append(k), df_lu['__count__'].append(_lu_[k])
    return pl.DataFrame(df_lu)
rt.histogram(pivot(becd.time_lu), bin_by='bin', count_by='__count__', w=512)